In [8]:
import gurobipy as gp
from gurobipy import *
import numpy as np

## Importing Data

In [9]:
# define function to download data
def load_data(url, filename):
    """
    Function to download and read the text file
    Arguments:
        url: link to download file
        filename: defined name to save data downloaded
    Return: 
        data: a list including text lines read from the text file
    """
    
    import urllib.request
    response = urllib.request.urlretrieve(url,filename)
    file = open(filename,'r')
    data = file.readlines()
    
    return data

In [10]:
# define function to parse the text file into problem variables
def parsing_wh(data):
    """
    Function to parse the text file
    Arguments:
        data: list of text lines to parse
    Return: tuple of variables including
        locations: number of warehouse locations
        capacity: list of each warehouse's capacity
        fixcost: list of each warehouse's fixed cost
        varcost: Numpy array sized (location, customer) with varcost[i,j] = average variable cost to serve 
                1 unit of demand from warehouse i to customer j
        customers: number of customer locations
        demand: list of each customer's demand        
    """
    
    import numpy as np
    
    [locations, customers] = [int(x) for x in data[0].split(' ')[1:-1]]
    capacity = []
    fixcost = []
    for i in range(locations):
        [c, f] = [float(x) for x in data[i+1].split(' ')[1:-1]]
        capacity.append(c)
        fixcost.append(f)
    demand = []
    varcost = np.zeros((locations,customers))
    for j in range(customers):
        row_step = 1 + int(np.ceil(locations/7))
        demand.append(float(data[j*row_step + (1+locations)].split(' ')[1]))
        v = list(np.concatenate([data[j*row_step + (1+locations+t)].split(' ')[1:-1] for t in range(1,row_step)]))
        for i in range(locations):
            varcost[i,j] = float(v[i])/demand[j] #cost of allocating demand of j to warehouse i per unit


    print('Locations:\n', locations)
    print('Capacities:\n', capacity)    
    print('Fixed costs:\n', fixcost)
    print('Variable costs:\n', varcost)
    
    print('Customers:\n', customers)
    print('Demands:\n', demand)
    
    return (locations, capacity, fixcost, varcost, customers, demand)

In [29]:
link = 'http://people.brunel.ac.uk/~mastjjb/jeb/orlib/files/cap114.txt'
file = 'cap114.txt'

data = load_data(link, file) 
(locations, capacity, fc, vc, customers, mu) = parsing_wh(data)

Locations:
 50
Capacities:
 [5000.0, 5000.0, 5000.0, 5000.0, 5000.0, 5000.0, 5000.0, 5000.0, 5000.0, 5000.0, 5000.0, 5000.0, 5000.0, 5000.0, 5000.0, 5000.0, 5000.0, 5000.0, 5000.0, 5000.0, 5000.0, 5000.0, 5000.0, 5000.0, 5000.0, 5000.0, 5000.0, 5000.0, 5000.0, 5000.0, 5000.0, 5000.0, 5000.0, 5000.0, 5000.0, 5000.0, 5000.0, 5000.0, 5000.0, 5000.0, 5000.0, 5000.0, 5000.0, 5000.0, 5000.0, 5000.0, 5000.0, 5000.0, 5000.0, 5000.0]
Fixed costs:
 [25000.0, 25000.0, 25000.0, 25000.0, 25000.0, 25000.0, 25000.0, 25000.0, 25000.0, 25000.0, 25000.0, 25000.0, 25000.0, 25000.0, 25000.0, 25000.0, 25000.0, 25000.0, 25000.0, 25000.0, 25000.0, 25000.0, 0.0, 25000.0, 25000.0, 25000.0, 25000.0, 25000.0, 25000.0, 25000.0, 25000.0, 25000.0, 25000.0, 25000.0, 25000.0, 25000.0, 25000.0, 25000.0, 25000.0, 25000.0, 25000.0, 25000.0, 25000.0, 25000.0, 25000.0, 25000.0, 25000.0, 25000.0, 25000.0, 25000.0]
Variable costs:
 [[17.875  27.225  56.725  ... 32.     69.05   35.9   ]
 [22.55   13.2    42.725  ... 32.1    

## Generating Scenario

In [35]:
def gen_scenario(N):
    np.random.seed(0)
    sigma = [np.random.uniform(0.2, 0.5) * m for m in mu]
    scenarios = []
    for i in range(N):
        scenario = []
        for j in range(len(mu)):
            scenario.append(round(np.random.normal(mu[j], sigma[j])))
        for j in range(len(scenario)):
            if scenario[j]<0:
                scenario[j] = 0
        scenarios.append(scenario)
        
    return scenarios

In [40]:
N = 1000
demand = gen_scenario(N)
scenarios = N

## Stochastic Programming

In [ ]:
m = gp.Model("Stochastic Warehouse Location Problem")
x = {}
y = {}

for i in range(locations):
    y[i] = m.addVar(vtype=GRB.BINARY, obj=fc[i])
    
for s in range(scenarios):
    for i in range(locations):
        for j in range(customers):
            x[i,j,s] = m.addVar(obj=vc[i][j]/N)
            
for s in range(scenarios):
    for j in range(customers):
        m.addConstr(sum(x[i,j,s] for i in range(locations))>=demand[s][j]) 
        
for s in range(scenarios):
    for i in range(locations):
        m.addConstr(sum(x[i,j,s] for j in range(customers)) <= capacity[i]*y[i])

m.optimize()

print("Objective: "+str(m.objVal))

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 100000 rows, 2500050 columns and 5050000 nonzeros
Model fingerprint: 0x4ec5057d
Variable types: 2500000 continuous, 50 integer (50 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+03]
  Objective range  [1e-03, 3e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+04]
Found heuristic solution: objective 3795378.9161
Presolve removed 301 rows and 15051 columns (presolve time = 5s) ...
Presolve removed 301 rows and 15051 columns
Presolve time: 9.82s
Presolved: 99699 rows, 2484999 columns, 5018900 nonzeros
Variable types: 2484950 continuous, 49 integer (49 binary)

Deterministic concurrent LP optimizer: primal simplex, dual simplex, and barrier
Showing barrier log only...

Root barrier log...

Ordering time: 1.16s

Barrier statistics:
 Dense cols : 49
 AA' NZ     : 2.534e+06
 Factor NZ  : 7.431e+06 (roughly